In [1]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import sys
from bert_serving.client import BertClient
from datetime import datetime
from nltk import tokenize

C:\Users\peter\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\peter\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\peter\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\peter\AppData\Roaming\Python\Python37\site-

In [46]:
#once terminal says "ready to serve request", connect
#may not be able to complete execution if the server isn't up
#avoid this because you might not be able to interrupt
bc = BertClient()

In [ ]:
#if want to close connection, run bc.close()
#even after terminating it will stay open if you don't close 
bc.close()

In [ ]:
#shut down server, don't forget
!bert-serving-terminate -port 5555

In [439]:
#import and sentence split all documents
raw = pd.read_excel('Final_Hackathon_1082Patients.xlsx')
raw.columns = ['id','note']
sentences = []
for i in range(len(raw['note'])):
    sentences.append(tokenize.sent_tokenize(raw['note'][i]))

raw['sent'] = sentences

#grab pickled embeddings

with open("sentembs_0.pkl", "rb") as fp:   # Unpickling
    sentembs_0 = pickle.load(fp)
    
with open("sentembs_1.pkl", "rb") as fp:   # Unpickling
    sentembs_1 = pickle.load(fp)
    
with open("sentembs_2.pkl", "rb") as fp:   # Unpickling
    sentembs_2 = pickle.load(fp)
    
with open("sentembs_3.pkl", "rb") as fp:   # Unpickling
    sentembs_3 = pickle.load(fp)
    
with open("sentembs_4.pkl", "rb") as fp:   # Unpickling
    sentembs_4 = pickle.load(fp)
    
with open("sentembs_5.pkl", "rb") as fp:   # Unpickling
    sentembs_5 = pickle.load(fp)
    
sentembs = sentembs_0
sentembs.extend(sentembs_1)
sentembs.extend(sentembs_2)
sentembs.extend(sentembs_3)
sentembs.extend(sentembs_4)
sentembs.extend(sentembs_5)

raw['sentembs'] = 0
raw['sentembs'] = sentembs

In [34]:
def check(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [441]:
check(raw['sentembs'][1][0],raw['sentembs'][1][1])

0.9127967

need to set some kind of threshold for similarity indicating a mention
will likely differ by criteria

functions to make:

args: full patient document, search sentence
output: list of sentences which meet threshold for similarity with search sentence
use: loop over each patient's document for each 

In [442]:
raw

,id,note,sent,sentembs
0,1081,new COVID-19 patient confirmed in Switzerland:...,[new COVID-19 patient confirmed in Switzerland...,"[[-0.42541587, -0.112368695, -0.12887241, 0.08..."
1,1080,new COVID-19 patient confirmed in Croatia: mal...,[new COVID-19 patient confirmed in Croatia: ma...,"[[-0.23784956, -0.085227765, -0.19446276, 0.17..."
2,1079,new COVID-19 patient confirmed in Algeria: mal...,[new COVID-19 patient confirmed in Algeria: ma...,"[[-0.5662999, -0.20431647, -0.10213997, 0.1434..."
3,1078,new COVID-19 patient confirmed in Afghanistan:...,[new COVID-19 patient confirmed in Afghanistan...,"[[-0.39275923, -0.018469699, -0.09146731, -0.0..."
4,1077,"new COVID-19 patient confirmed in Austria: 24,...",[new COVID-19 patient confirmed in Austria: 24...,"[[-0.44372812, -0.02232341, -0.27159494, 0.046..."
...,...,...,...,...
1076,5,new confirmed imported COVID-19 pneumonia in T...,[new confirmed imported COVID-19 pneumonia in ...,"[[-0.027217222, -0.3362413, -0.12997176, 0.195..."
1077,4,First confirmed imported cases in Zhejiang: pa...,[First confirmed imported cases in Zhejiang: p...,"[[-0.29076847, -0.4798644, -0.32880166, 0.1344..."
1078,3,First confirmed imported COVID-19 pneumonia pa...,[First confirmed imported COVID-19 pneumonia p...,"[[-0.084986426, -0.31383824, -0.27918404, 0.12..."
1079,2,First confirmed imported COVID-19 pneumonia pa...,[First confirmed imported COVID-19 pneumonia p...,"[[-0.2739502, -0.39795566, -0.3605169, 0.13827..."


In [443]:
#finds sentences in document that pass provided similarity threshold
#returns list of sentences ordered by similarity, with second item in each sub list being the sentence index for that document
def check_multi(df,pind,search_emb,threshold):
    #dataframe containing id, note, sentences, sentence embeddings
    #patient index
    #search sentence embedding
    #minimum similarity score
    emb = df['sentembs'][pind]
    sent = df['sent'][pind]
    mentions = []
    positions = []
    for i in range(len(emb)):
        a = check(emb[i],search_emb)
        if a > threshold:
            mentions.append([sent[i],a])
            positions.append([i,a])
    mentions = sorted(mentions, key=lambda x: x[1], reverse = True)
    positions = sorted(positions, key=lambda x: x[1], reverse = True)
    
    if len(mentions) > 3:        
        mentions = mentions[0:3]
        positions = positions[0:3]
        
    keep_mentions = []
    for i in range(len(mentions)):
        keep_mentions.append(mentions[i][0])
        
    keep_positions = []
    for i in range(len(positions)):
        keep_positions.append(positions[i][0])
        
    return keep_mentions, keep_positions

In [444]:
raw['note'][0]

"new COVID-19 patient confirmed in Switzerland: male, 70, infected in Milan during an event 2/15, symptom onset 2/17 service:  blue general surgery  history of the present illness:  mrs.  is a patient with a prior medical history including hypothyroidism, iron-deficiency anemia, osteoporosis, increased cholesterol, peptic ulcer disease, chronic constipation, and status post appendectomy as well as status post hysterectomy and bilateral salpingo-oophorectomy.  she was transferred from an outside hospital for evaluation and treatment of adenocarcinoma involving the transverse colon and gallbladder.  at the outside hospital the patient had already seemed some workup for a couple weeks duration of bilateral upper quadrant pain which occasionally radiated to her back. she was also complaining of fatigue for two weeks but a good appetite and the patient denied weight loss, melena, or bright red blood per rectum.  a cat scan of the abdomen showed air-fluid levels in the gallbladder.  it also 

In [460]:
start = datetime.now()
    
search_emb = bc.encode(['The patient has provided informed consent.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .93)[0])
raw['c4_1'] = catch_mentions

search_emb = bc.encode(['Informed consent was provided by the patient.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .93)[0])
raw['c4_2'] = catch_mentions

search_emb = bc.encode(['The patient has agreed to participate in the study.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .93)[0])
raw['c4_3'] = catch_mentions

raw['c4_sentence'] = raw['c4_1'] + raw['c4_2'] + raw['c4_3']
raw = raw.drop(['c4_1','c4_2','c4_3'], axis = 1)

end = datetime.now()
print(end - start)

raw['c4_sentence'].sum()

0:00:52.438705


['in the interim, the patient has been given an event recorder.',
 'conscious sedation:  was provided with appropriate monitoring performed by a member of the nursing staff.',
 'patient was preopped and consented for surgery.',
 'patient was preopped and consented for surgery.',
 'conscious sedation:  was provided with appropriate monitoring performed by a member of the nursing staff.',
 'he was prepared and consented as per standard.',
 'he was prepared and consented as per standard.',
 'doctor  was informed of these pending results.',
 'he was prepared and consented for surgery as per standard.',
 'he was prepared and consented for surgery as per standard.',
 'the family agreed and consented to perform surgery.',
 'the family agreed and consented to perform surgery.',
 'the family agreed and consented to perform surgery.',
 'the patient was seen by cardiac surgery at which time possible risks and complications were explained to him and consent was signed.',
 'the patient was seen by 

In [461]:
start = datetime.now()

search_emb = bc.encode(['The patient is allergic to a drug.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .945)[0])
raw['c8_1'] = catch_mentions

search_emb = bc.encode(['The patient an allergic reaction to the prescribed drug.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .945)[0])
raw['c8_2'] = catch_mentions

search_emb = bc.encode(['An allergic reaction was noted.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .945)[0])
raw['c8_3'] = catch_mentions

raw['c8_sentence'] = raw['c8_1'] + raw['c8_2'] + raw['c8_3']
raw = raw.drop(['c8_1','c8_2','c8_3'], axis = 1)

end = datetime.now()
print(end - start)

raw['c8_sentence'].sum()

0:00:52.273533


['allergies:  the patient has no known drug allergies.',
 'a new allergy to dilantin was documented.',
 'a new allergy to dilantin was documented.',
 'the patient works at a fast food restaurant.',
 'allergies:  the patient has no known drug allergies.',
 'allergies:  the patient has no known drug allergies.',
 'allergies:  the patient has no known drug allergies.',
 'allergies:  the patient has no known drug allergies.',
 'allergies:  the patient has no known drug allergies.',
 'allergies:  the patient has an allergy to bactrim.',
 'allergies:  the patient has an allergy to bactrim.',
 'allergies:  the patient has a questionable allergy to bactrim which causes edema.',
 'allergies:  the patient has a questionable allergy to bactrim which causes edema.',
 'allergies:  he is allergic to sulfa, causes facial edema.',
 'allergies:  the patient has no known drug allergies.',
 'allergies:  the patient has no known drug allergies.',
 'allergies:   to sulfa which causes a rash.',
 'allergies:

In [462]:
start = datetime.now()

search_emb = bc.encode(['The patient has HIV.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .955)[0])
raw['c9_1'] = catch_mentions

search_emb = bc.encode(['The patient has human immunodeficiency virus.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .93)[0])
raw['c9_2'] = catch_mentions

raw['c9_sentence'] = raw['c9_1'] + raw['c9_2']
raw = raw.drop(['c9_1','c9_2'], axis = 1)

end = datetime.now()
print(end - start)

raw['c9_sentence'].sum()

0:00:34.763292


['the patient has chronic renal insufficiency.',
 'past medical history:   human immunodeficiency virus; diagnosed in .',
 'he was continued on his antiretroviral therapy for his human immunodeficiency virus and dapsone for prophylaxis.',
 'infectious disease:  the patient was closely followed by the infectious disease service during this hospitalization given his human immunodeficiency virus disease.',
 'history of present illness:  the patient is a 36-year-old male with human immunodeficiency virus who developed fever and nausea several days prior to admission.',
 'human immunodeficiency virus antibody test was negative.',
 'human immunodeficiency virus issues:  the patient was restarted on her highly active antiretroviral therapy regimen on hospital day four.']

In [463]:
start = datetime.now()

search_emb = bc.encode(['The patient has not been diagnosed with hepatitis C, chronic hepatitis B, or hepatic disease.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .94)[0])
raw['c10_sentence'] = catch_mentions

end = datetime.now()
print(end - start)

raw['c10_sentence'].sum()

0:00:17.623363


['his hepatitis serologies were negative.',
 'in regard to his cirrhosis that was documented from abdominal ultrasound , the patient does have evidence of chronic lft elevation, but no coagulopathy.',
 'also in the  intensive care unit, he had a hepatitis screen that showed him positive for hepatitis c antibody and negative hepatitis a or hepatitis b antibodies.',
 'the patient had no evidence of diabetic ketoacidosis during his hospitalization.',
 '# hepatitis c: she showed no signs of decompensation.',
 'past medical history:  chronic hepatitis c with history of hepatic encephalopathy.',
 'family history: no family history of liver disease, diabetes, or cardiovascular  disease.',
 'the patient also has cirrhosis of unclear etiology.',
 'his mother has a history of atherosclerosis and died at the age of   no known history of cancer or clotting disorders in the family.',
 'past medical history:  consistent with cirrhosis, hepatitis c, hepatocellular carcinoma.',
 'patient did not have 

In [464]:
start = datetime.now()

search_emb = bc.encode(['The patient has no history of opioid use.'])
catch_mentions = []
for i in range(len(raw)):
    catch_mentions.append(check_multi(raw, i, search_emb, .945)[0])
raw['c12_sentence'] = catch_mentions

end = datetime.now()
print(end - start)

raw['c12_sentence'].sum()

0:00:17.126686


['she denies any tobacco use or intravenous drug use.',
 'past medical history:   the patient had no significant past medical history.',
 'she did not receive any nitrates or morphine at the time.',
 'social history:  the patient has a distant smoking history and only occasional alcohol usage.',
 'substance abuse:  the patient had cocaine intoxication on presentation and has a long history of alcohol dependence.',
 'she denied any current or remote history of tobacco use.',
 'she has a history of tobacco use and occasionally drinks alcoholic beverages.',
 'he denies a history of heavy alcohol use.',
 'he has a remote history of tobacco use.',
 'the patient has no history of diabetes.',
 'the patient has a history of marijuana use.',
 'he denies any intravenous drug use and has used marijuana in the past.',
 'the patient denied alcohol or other drug use.',
 'the patient has never had these symptoms in the past.',
 'he had no tobacco history and no use of alcohol.',
 'he had no known dru

In [472]:
#pickling raw
with open("sentence_comparison_extraction.pkl", "wb") as fp:   #Pickling
    pickle.dump(raw, fp)

In [532]:
'no' in word_tokenize('hi im peter')

False

In [533]:
#need to tokenize 'hi im peter'.split()

def classify(listed):
    
    negation = ['no', 'not', 'nor', 'non', 'none', 'never', 'without', 'neither', 'deny', 
                'denies', 'reject', 'rejects', 'subside', 'subsided', 'negative']
    uncertainty = ['uncertain', 'unclear', 'indeterminate']        
    
    mention_classifications = []
    for i in range(len(listed)):
        mentions = listed[i]
        mentions = mentions.lower()
        mentions = word_tokenize(mentions)
        
        for ii in range(len(uncertainty)):
            uncertain = uncertainty[ii] in mentions
            classification = 1
            if uncertain == True:
                classification = 0
                break

        for ii in range(len(negation)):
            negatory = negation[ii] in mentions
            if negatory == True:
                classification = -1
                break
        
        mention_classifications.append(classification)
    
    return mention_classifications      

In [505]:
#need to flip for negation on some
#negation on 8 gives 1
#negation on 9 gives 1
#negation on 10 gives 1
#negation on 12 gives 1

def flip (listed):
    mention_classifications = []
    for i in range(len(listed)):
        mention = listed[i] * -1
        mention_classifications.append(mention)
        
    return mention_classifications

In [510]:
flip([0,1])

[0, -1]

In [588]:
def aggregate(listed):
    
    aggregation = 99
    if len(listed) == 0:
        aggregation = 0
    else:
        if sum(listed) == 0:
            aggregation = 1
        else:
            aggregation = sum(listed)
            if aggregation >= 1:
                aggregation = 1
            if aggregation <= -1:
                aggregation = -1
    
    return aggregation 

In [587]:
aggregate([0,-1])

-1

In [ ]:
raw

In [602]:
raw

,id,note,sent,sentembs,c4_sentence,c8_sentence,c9_sentence,c10_sentence,c12_sentence
0,1081,new COVID-19 patient confirmed in Switzerland:...,[new COVID-19 patient confirmed in Switzerland...,"[[-0.42541587, -0.112368695, -0.12887241, 0.08...",[],[],[],[],[]
1,1080,new COVID-19 patient confirmed in Croatia: mal...,[new COVID-19 patient confirmed in Croatia: ma...,"[[-0.23784956, -0.085227765, -0.19446276, 0.17...",[],[],[],[],[]
2,1079,new COVID-19 patient confirmed in Algeria: mal...,[new COVID-19 patient confirmed in Algeria: ma...,"[[-0.5662999, -0.20431647, -0.10213997, 0.1434...",[],[],[],[],[]
3,1078,new COVID-19 patient confirmed in Afghanistan:...,[new COVID-19 patient confirmed in Afghanistan...,"[[-0.39275923, -0.018469699, -0.09146731, -0.0...",[],[],[],[],[]
4,1077,"new COVID-19 patient confirmed in Austria: 24,...",[new COVID-19 patient confirmed in Austria: 24...,"[[-0.44372812, -0.02232341, -0.27159494, 0.046...",[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...
1076,5,new confirmed imported COVID-19 pneumonia in T...,[new confirmed imported COVID-19 pneumonia in ...,"[[-0.027217222, -0.3362413, -0.12997176, 0.195...",[we have telephoned (doctor with this report ...,[],[],[],[]
1077,4,First confirmed imported cases in Zhejiang: pa...,[First confirmed imported cases in Zhejiang: p...,"[[-0.29076847, -0.4798644, -0.32880166, 0.1344...",[],[],[],[],[]
1078,3,First confirmed imported COVID-19 pneumonia pa...,[First confirmed imported COVID-19 pneumonia p...,"[[-0.084986426, -0.31383824, -0.27918404, 0.12...",[],[],[human immunodeficiency virus issues: the pat...,[],[]
1079,2,First confirmed imported COVID-19 pneumonia pa...,[First confirmed imported COVID-19 pneumonia p...,"[[-0.2739502, -0.39795566, -0.3605169, 0.13827...",[],[allergies: the patient has no known drug all...,[],[],[]


In [534]:
classify(['no','john','peter','Apple denies any wrongdoing','spasms were detected in the heart','lung status Indeterminate'])

[-1, 1, 1, -1, 1, 0]

In [466]:
classify(['no','john','peter','Apple denies any wrongdoing','spasms were detected in the heart','lung status Indeterminate'])

[-1, 1, 1, -1, 1, 0]

In [467]:
classify(['hello johnny appleseed'])

[1]

In [468]:
classify([])

[]

In [603]:
sentence_phase_2 = raw[['id','note']].copy()
sentence_phase_2['c4_sentence'] = raw['c4_sentence'].copy().apply(classify)
sentence_phase_2['c8_sentence'] = raw['c8_sentence'].copy().apply(classify).apply(flip)
sentence_phase_2['c9_sentence'] = raw['c9_sentence'].copy().apply(classify).apply(flip)
sentence_phase_2['c10_sentence'] = raw['c10_sentence'].copy().apply(classify).apply(flip)
sentence_phase_2['c12_sentence'] = raw['c12_sentence'].copy().apply(classify).apply(flip)

sentence_phase_3 = sentence_phase_2[['id']].copy()
sentence_phase_3['c4_sentence'] = sentence_phase_2['c4_sentence'].copy().apply(aggregate)
sentence_phase_3['c8_sentence'] = sentence_phase_2['c8_sentence'].copy().apply(aggregate)
sentence_phase_3['c9_sentence'] = sentence_phase_2['c9_sentence'].copy().apply(aggregate)
sentence_phase_3['c10_sentence'] = sentence_phase_2['c10_sentence'].copy().apply(aggregate)
sentence_phase_3['c12_sentence'] = sentence_phase_2['c12_sentence'].copy().apply(aggregate)
sentence_phase_3['c11_sentence'] = sentence_phase_3['c10_sentence']

In [590]:
sentence_phase_2

,id,note,c4_sentence,c8_sentence,c9_sentence,c10_sentence,c12_sentence
0,1081,new COVID-19 patient confirmed in Switzerland:...,[],[],[],[],[]
1,1080,new COVID-19 patient confirmed in Croatia: mal...,[],[],[],[],[]
2,1079,new COVID-19 patient confirmed in Algeria: mal...,[],[],[],[],[]
3,1078,new COVID-19 patient confirmed in Afghanistan:...,[],[],[],[],[]
4,1077,"new COVID-19 patient confirmed in Austria: 24,...",[],[],[],[],[]
...,...,...,...,...,...,...,...
1076,5,new confirmed imported COVID-19 pneumonia in T...,[1],[],[],[],[]
1077,4,First confirmed imported cases in Zhejiang: pa...,[],[],[],[],[]
1078,3,First confirmed imported COVID-19 pneumonia pa...,[],[],[-1],[],[]
1079,2,First confirmed imported COVID-19 pneumonia pa...,[],[1],[],[],[]


In [604]:
sentence_phase_3

,id,c4_sentence,c8_sentence,c9_sentence,c10_sentence,c12_sentence,c11_sentence
0,1081,0,0,0,0,0,0
1,1080,0,0,0,0,0,0
2,1079,0,0,0,0,0,0
3,1078,0,0,0,0,0,0
4,1077,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1076,5,1,0,0,0,0,0
1077,4,0,0,0,0,0,0
1078,3,0,0,-1,0,0,0
1079,2,0,1,0,0,0,0


In [520]:
raw['c8_sentence'][1079]

['allergies:  the patient has no known drug allergies.']

In [518]:
raw['c9_sentence'][1078]

['human immunodeficiency virus issues:  the patient was restarted on her highly active antiretroviral therapy regimen on hospital day four.']

In [478]:
with open("bert_qa_results.pkl", "rb") as fp:   # Unpickling
    qa = pickle.load(fp)
    
qa['id'] = qa['id'].astype('int')

In [ ]:
def classify(listed):
    
    negation = ['no', 'not', 'nor', 'non', 'none', 'never', 'without', 'neither', 'deny', 
                'denies', 'reject', 'rejects', 'subside', 'subsided', 'negative']
    uncertainty = ['uncertain', 'unclear', 'indeterminate']        
    
    mention_classifications = []
    for i in range(len(listed)):
        mentions = listed[i]
        mentions = mentions.lower()
        mentions = word_tokenize(mentions)
        
        for ii in range(len(uncertainty)):
            uncertain = uncertainty[ii] in mentions
            classification = 1
            if uncertain == True:
                classification = 0
                break

        for ii in range(len(negation)):
            negatory = negation[ii] in mentions
            if negatory == True:
                classification = -1
                break
        
        mention_classifications.append(classification)
    
    return mention_classifications      

In [605]:
qa_phase_2 = qa[['id']].copy()
qa_phase_2['c4_qa'] = qa['c4_qa'].copy().apply(classify)
qa_phase_2['c8_qa'] = qa['c8_qa'].copy().apply(classify).apply(flip)
qa_phase_2['c9_qa'] = qa['c9_qa'].copy().apply(classify).apply(flip)
qa_phase_2['c10_qa'] = qa['c10_qa'].copy().apply(classify).apply(flip)
qa_phase_2['c12_qa'] = qa['c12_qa'].copy().apply(classify).apply(flip)

qa_phase_3 = qa_phase_2[['id']].copy()
qa_phase_3['c4_qa'] = qa_phase_2['c4_qa'].copy().apply(aggregate)
qa_phase_3['c8_qa'] = qa_phase_2['c8_qa'].copy().apply(aggregate)
qa_phase_3['c9_qa'] = qa_phase_2['c9_qa'].copy().apply(aggregate)
qa_phase_3['c10_qa'] = qa_phase_2['c10_qa'].copy().apply(aggregate)
qa_phase_3['c12_qa'] = qa_phase_2['c12_qa'].copy().apply(aggregate)
qa_phase_3['c11_qa'] = qa_phase_3['c10_qa']

In [593]:
qa_phase_2

,id,c4_qa,c8_qa,c9_qa,c10_qa,c12_qa
0,1081,[],[],[],[],[]
1,1080,[],[-1],[],[],[]
2,1079,[],[-1],[],[],[]
3,1078,[],[-1],[],[],[]
4,1077,[],[1],[],[],[]
...,...,...,...,...,...,...
1076,5,[],[],[],[],[]
1077,4,[],[1],[],[],[]
1078,3,[],[],[],[],[]
1079,2,[],[],[],[],[]


In [606]:
qa_phase_3

,id,c4_qa,c8_qa,c9_qa,c10_qa,c12_qa,c11_qa
0,1081,0,0,0,0,0,0
1,1080,0,-1,0,0,0,0
2,1079,0,-1,0,0,0,0
3,1078,0,-1,0,0,0,0
4,1077,0,1,0,0,0,0
...,...,...,...,...,...,...,...
1076,5,0,0,0,0,0,0
1077,4,0,1,0,0,0,0
1078,3,0,0,0,0,0,0
1079,2,0,0,0,0,0,0


In [540]:
with open("w2v_results.pkl", "rb") as fp:   # Unpickling
    word = pickle.load(fp)
    
#word has default 0 vote on c1_word, c2_word, c3_word, c5_word, c6_word, c7_word, c11_word
word = word[['id','c4_word','c8_word','c9_word','c10_word','c12_word']]

In [607]:
word_phase_2 = word[['id']].copy()
word_phase_2['c4_word'] = word['c4_word'].copy().apply(classify)
word_phase_2['c8_word'] = word['c8_word'].copy().apply(classify).apply(flip)
word_phase_2['c9_word'] = word['c9_word'].copy().apply(classify).apply(flip)
word_phase_2['c10_word'] = word['c10_word'].copy().apply(classify).apply(flip)
word_phase_2['c12_word'] = word['c12_word'].copy().apply(classify).apply(flip)

word_phase_3 = word_phase_2[['id']].copy()
word_phase_3['c4_word'] = word_phase_2['c4_word'].copy().apply(aggregate)
word_phase_3['c8_word'] = word_phase_2['c8_word'].copy().apply(aggregate)
word_phase_3['c9_word'] = word_phase_2['c9_word'].copy().apply(aggregate)
word_phase_3['c10_word'] = word_phase_2['c10_word'].copy().apply(aggregate)
word_phase_3['c12_word'] = word_phase_2['c12_word'].copy().apply(aggregate)
word_phase_3['c11_word'] = word_phase_3['c10_word']

In [596]:
word_phase_2

,id,c4_word,c8_word,c9_word,c10_word,c12_word
0,1081,[],[],[],[-1],[]
1,1080,[],"[-1, -1]",[],[],[]
2,1079,[],"[-1, -1]",[],[],[]
3,1078,[],"[-1, -1]",[],[],[]
4,1077,[],"[1, 1, 1, 1]",[],"[-1, -1, -1, 1, -1, -1, -1]",[]
...,...,...,...,...,...,...
1076,5,[],"[-1, -1]",[],[-1],[]
1077,4,[],"[1, 1, 1, 1]",[],[],[]
1078,3,[],"[1, -1, 1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]",[],[]
1079,2,[],"[-1, 1, -1, 1]",[],[],[]


In [608]:
word_phase_3

,id,c4_word,c8_word,c9_word,c10_word,c12_word,c11_word
0,1081,0,0,0,-1,0,-1
1,1080,0,-1,0,0,0,0
2,1079,0,-1,0,0,0,0
3,1078,0,-1,0,0,0,0
4,1077,0,1,0,-1,0,-1
...,...,...,...,...,...,...,...
1076,5,0,-1,0,-1,0,-1
1077,4,0,1,0,0,0,0
1078,3,0,1,-1,0,0,0
1079,2,0,1,0,0,0,0


In [ ]:
raw['note'][2]

In [626]:
def bmi(document):
    
    unhealthy = ['overweight','obese','anorexic','obesity','anorexia']
    
    words = word_tokenize(document)
    
    for i in range(len(unhealthy)):
        outside = unhealthy[i] in words
        classification = 1
        if outside == True:
            classification = 0
            break
    
    return classification    

In [ ]:
raw['c5'] = raw['note'].apply(bmi)
raw['c6'] = raw['c5']
raw['c7'] = raw['c5']

In [667]:
with open("regex_df.pkl", "rb") as fp:   # Unpickling
    regex = pickle.load(fp)

regex['c1_regex'] = regex['c1_re'].astype('int')
regex['c2_regex'] = regex['c2_re'].astype('int')
regex['c3_regex'] = regex['c3_re'].astype('int')
regex['c4_regex'] = regex['c4_re'].astype('int')
regex['c8_regex'] = regex['c8_re'].astype('int')
regex_phase_3 = regex.drop(['c1_re','c2_re','c3_re','c4_re','c8_re'], axis = 1)

regex_phase_3['c1_regex'] = regex_phase_3['c1_regex'].replace(0, -1)
regex_phase_3['c2_regex'] = regex_phase_3['c2_regex'].replace(0, -1)
regex_phase_3['c3_regex'] = regex_phase_3['c3_regex'].replace(0, -1)
regex_phase_3['c4_regex'] = regex_phase_3['c4_regex'].replace(0, -1)
regex_phase_3['c8_regex'] = regex_phase_3['c8_regex'].replace(0, -1)

In [668]:
regex_phase_3

,id,c1_regex,c2_regex,c3_regex,c4_regex,c8_regex
0,1081,1,1,1,-1,-1
1,1080,1,-1,1,-1,-1
2,1079,1,-1,1,-1,-1
3,1078,1,1,-1,-1,-1
4,1077,1,-1,-1,-1,-1
...,...,...,...,...,...,...
1076,5,1,1,-1,-1,-1
1077,4,-1,1,1,-1,-1
1078,3,1,1,-1,-1,-1
1079,2,1,1,1,-1,-1


In [673]:
voter.columns

Index(['id', 'note', 'c5', 'c6', 'c7', 'c4_sentence', 'c8_sentence',
       'c9_sentence', 'c10_sentence', 'c12_sentence', 'c11_sentence',
       'c1_regex', 'c2_regex', 'c3_regex', 'c4_regex', 'c8_regex', 'c4_word',
       'c8_word', 'c9_word', 'c10_word', 'c12_word', 'c11_word', 'c4_qa',
       'c8_qa', 'c9_qa', 'c10_qa', 'c12_qa', 'c11_qa'],
      dtype='object')

In [687]:
def vote(my_vote):
    if my_vote < 0:
        result = 0
    else:
        result = 1
    return result

In [718]:
#voting: sum the votes from each: -1, 0, or 1.
#sum < 0 returns 0
#sum >= 0 returns 1.
voter = raw[['id','note','c5','c6','c7']].copy()
voter = voter.join(sentence_phase_3.drop(['id'], axis = 1))
voter = voter.join(regex_phase_3.drop(['id'], axis = 1))
voter = voter.join(word_phase_3.drop(['id'], axis = 1))
voter = voter.join(qa_phase_3.drop(['id'], axis = 1))

voter['c1'] = voter['c1_regex']
voter['c2'] = voter['c2_regex']
voter['c3'] = voter['c3_regex']
voter['c4'] = voter['c4_regex'] + voter['c4_word'] + voter['c4_sentence'] + voter['c4_qa']
#voter['c5'] already defined by bmi from raw
#voter['c6'] already defined by bmi from raw
#voter['c7'] already defined by bmi from raw
voter['c8'] = voter['c8_regex'] + voter['c8_word'] + voter['c8_sentence'] + voter['c8_qa']
voter['c9'] = voter['c9_word'] + voter['c9_sentence'] + voter['c9_qa']
voter['c10'] = voter['c10_word'] + voter['c10_sentence'] + voter['c10_qa']
voter['c11'] = voter['c11_word'] + voter['c11_sentence'] + voter['c11_qa']
voter['c12'] = voter['c12_word'] + voter['c12_sentence'] + voter['c12_qa']

voter['c1'] = voter['c1'].map(vote)
voter['c2'] = voter['c2'].map(vote)
voter['c3'] = voter['c3'].map(vote)
voter['c4'] = voter['c4'].map(vote)
voter['c5'] = voter['c5'].map(vote)
voter['c6'] = voter['c6'].map(vote)
voter['c7'] = voter['c7'].map(vote)
voter['c8'] = voter['c8'].map(vote)
voter['c9'] = voter['c9'].map(vote)
voter['c10'] = voter['c10'].map(vote)
voter['c11'] = voter['c11'].map(vote)
voter['c12'] = voter['c12'].map(vote)

final = voter[['id','note','c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12']].copy()

final['eligibility'] = final['c1'] * final['c2'] *  final['c3'] * final['c4'] * final['c5'] * final['c6'] * final['c7'] * final['c8'] * final['c9'] * final['c10'] * final['c11'] * final['c12']

In [736]:
final[final['eligibility'] == 1]

,id,note,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,eligibility
283,798,"new confirmed COVID-19 patient in Germany: 52,...",1,1,1,1,1,1,1,1,1,1,1,1,1
728,353,new confirmed COVID-19 patient in Japan: Hokka...,1,1,1,1,1,1,1,1,1,1,1,1,1
790,291,"new confirmed COVID-19 patient in Japan: male,...",1,1,1,1,1,1,1,1,1,1,1,1,1
818,263,"new confirmed COVID-19 patient in Japan: male,...",1,1,1,1,1,1,1,1,1,1,1,1,1
868,213,new confirmed COVID-19 patient in France: Pari...,1,1,1,1,1,1,1,1,1,1,1,1,1
1080,1,"new confirmed COVID-19 patient in Tyumen, Russ...",1,1,1,1,1,1,1,1,1,1,1,1,1


In [720]:
final['eligibility'].sum()

6

In [728]:
answers = final[final['eligibility'] == 1]['note'].copy().reset_index().drop('index',axis=1)

In [733]:
answers['note'][0]

'new confirmed COVID-19 patient in Germany: 52, male, works at same company as patients 1-5 service: neurosurgery  allergies: patient recorded as having no known allergies to drugs  attending: chief complaint: referred after ct head done in am; headaches for 2-3 months   major surgical or invasive procedure: bilateral craniotomies with evacuation of subdural hematomas   history of present illness: this is a 66 year old  who was referred by his pcp for  head ct the morning of admission on the  of . the history is as per the interpreter (pt only speaks cantonese). the patient describes a bilateral headache of  months duration, partially relieved by pain medication (tylenol). he does not recall any incident of trauma or any falls, but does describe a routine daily chinese exercise where he shakes his head to either side. the headache has been constant, and last night, increased in intensity and the patient describes a feeling of loss of balance (ataxia). he has has no visual field disturb

In [734]:
answers['note'][1]

'new confirmed COVID-19 patient in Japan: Hokkaido, male, 70s service: orthopaedics  allergies: patient recorded as having no known allergies to drugs  attending: chief complaint: s/p mvc with injuries  major surgical or invasive procedure: :  traction pin to lle :  im nail right femur, closed reduction right pilon fracture, application multiplanar external fixator, operative treatment of left subtrochanteric femur fracture with intramedullary nail, washout and debridement, left open femur fracture wound, treatment of left femoral shaft fracture with im implant. :  ivc filter placement, i&d left open femur fx :  orif right pilon fx   history of present illness: mr.  is a 33 year old man who was invoved in a high speed rollover motor vehicle crash on .  he was trapped under his car for 30 minutes with a gcs of 3  he was taken by  to  for further care and treatment.  past medical history: denies  social history: works as a forklift operator lives with wife  family history: n/a  physical 

In [737]:
answers['note'][2]

'new confirmed COVID-19 patient in Japan: male, 60s, lives in Aichi Prefecture, contact with case 44 and case 53 service: neurosurgery  allergies: patient recorded as having no known allergies to drugs  attending: chief complaint: headache  major surgical or invasive procedure: suboccipital decompressive craniotomy, c1 laminectomy, duraplasty   history of present illness: the patient is a 42-year-old patient who is well known to the  neurosurgical service. he has been seen in doctor office multiple times on an outpatient basis. he has been diagnosed with a chiari ii malformation in the setting of a previously operated spinal meningeal viral series. the patient has classic signs of headaches that are positional and aggravated by coughing. the patient has been worked up with an mri scan that shows the tight posterior fossa with inferior displacement of parts of the cerebellum to the level of c he was counseled. he wished to proceed with elective decompression today. the patient was conse

In [738]:
answers['note'][3]

"new confirmed COVID-19 patient in Japan: male, 70s, symptom onset 2/1 (cold, fever), visited medical institution 2/6 service: medicine  allergies: patient recorded as having no known allergies to drugs  attending: chief complaint: black stools and weakness x 7 days.  major surgical or invasive procedure: esophagogastroduodenoscopy   history of present illness: 87 yo f w/ h/o chf, htn, afib, and h/o recent fall resulting in a compression  transferred from  for c/o black stools w/ hct 3 and inr 3 due to no available icu beds at . patient was af, bp 130/44 and hr  she received 2 u prbc, 40 iv protonix, and 10 mg iv vit k for this. on arrival to  er, hct 2 and inr  bp 150/54 and hr  patient received additional 5 mg sq vit k, in addition to 2 u ffp. ng lavage was done and yielded coffee ground emesis w/ the first 250 cc, followed by a pink-tinged fluid with the second 250 cc. on hx patient reports no po x 2 weeks due to nausea w/o c/o pain w/ eating. she has never had black stools in the p

In [739]:
answers['note'][4]

'new confirmed COVID-19 patient in France: Paris, contact of case 15 service: cardiothoracic  allergies: patient recorded as having no known allergies to drugs  attending: chief complaint: 67m with fever and cough  major surgical or invasive procedure:  cabg x 4 (lad, pda, , ) cardiac cath   history of present illness: pt is a 67yo homeless man with pmh sig for "enlarged heart" who presents to the ed by ems complaining of fever/chills and productive cough with progressively worsening sob over the past 3-4 days. denies palp/n/v.  has had diaphoresis.  no orthopnea/pnd. in the  pt had increased o2 requirements to 100% nrb, given solumedrol, ceftriaxone, azithromycin.  given elevated cardiac enzymes, ekg changes pt started on heparin drip.   past medical history: ?cardiomegaly knee pain  social history: +smoker former golf pro homeless + former alcohol use -  quit 7 yrs ago no ivda  family history: unable to obtain  physical exam: t 9  hr 98  bp 70/50 ac 500x18 fio2 100% rr 20 gen: using 

In [740]:
answers['note'][5]

"new confirmed COVID-19 patient in Tyumen, Russia:  citizens service: med  allergies: patient recorded as having no known allergies to drugs  attending: chief complaint: inability of extended care facility to administer meds  major surgical or invasive procedure:  after the risks and benefits of the procedure were discussed with the patient and informed consent was obtained, the patient was placed prone on the angiography table. her left flank was prepped and draped in the standard sterile fashion. gastrografin swallow evaluation video esophageal evalaution ct guided aspiration of prevertebral fluid collection   history of present illness: this is a 74 year-old man with a bmi of 29, and history of  syphylis, hypertension  who suffered a fall from wheelchair at  home on , mri at osh revealed c3-c4 area suspicious for osteomyletis, sent to . c-spine and x-ray at that time showed no pathology. he was recently discharged from  () to rehab after findings here including c3-c5  / ?staph osteo